In [60]:
import json
import tweepy
import csv

twitter_cred = dict()

twitter_cred['CONSUMER_KEY'] = '17XHiftb4Iu5hVAIh4NacWj9S'
twitter_cred['CONSUMER_SECRET'] = '8xAXhFsfnkiFjSFO5THqRO5J8x8lKSzC9J2GNPUSY7ZPb5z6Qv'
twitter_cred['ACCESS_KEY'] = '1392379572-FInzXaziEqyNCGc8jqtU9LwnFFlHYRpjoihZpiX'
twitter_cred['ACCESS_SECRET'] = 'JBDveSiUU6aXpwx8QkxYhXloDr9DyHn7ocqrqgWqwqctw'

In [61]:
with open('twitter_credentials.json', 'w') as secret_info:
    json.dump(twitter_cred, secret_info, indent=4, sort_keys=True)

In [62]:
with open('twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

# Create the api endpoint

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [63]:
maximum_number_of_tweets_to_be_extracted = \
int(input('Enter the number of tweets that you want to extract- '))

Enter the number of tweets that you want to extract- 10


In [64]:
hashtag = input('Enter the hashtag you want to scrape- ')

Enter the hashtag you want to scrape- Datascience


In [65]:
import re
import sys

def handle_emojis(tweet):
    # Smile -- :), : ), :-), (:, ( :, (-:, :')
    tweet = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', tweet)
    # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    tweet = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', tweet)
    # Love -- <3, :*
    tweet = re.sub(r'(<3|:\*)', ' EMO_POS ', tweet)
    # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    tweet = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', tweet)
    # Sad -- :-(, : (, :(, ):, )-:
    tweet = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', tweet)
    # Cry -- :,(, :'(, :"(
    tweet = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', tweet)
    return tweet

In [66]:
def preprocess_tweet(tweet):
    processed_tweet = []
    # Replaces URLs with the word URL
    tweet = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', tweet)
    # Replace @handle with the word USER_MENTION
    tweet = re.sub(r'@[\S]+', 'USER_MENTION', tweet)
    # Replaces #hashtag with hashtag
    tweet = re.sub(r'#(\S+)', r' \1 ', tweet)
    # Remove RT (retweet)
    tweet = re.sub(r'\brt\b', '', tweet)
    # Replace 2+ dots with space
    tweet = re.sub(r'\.{2,}', ' ', tweet)
    # Strip space, " and ' from tweet
    tweet = tweet.strip(' "\'')
    # Replace emojis with either EMO_POS or EMO_NEG
    tweet = handle_emojis(tweet)
    # Replace multiple spaces with a single space
    tweet = re.sub(r'\s+', ' ', tweet)
    words = tweet.split()

    for word in words:
        word = preprocess_word(word)
        if is_valid_word(word):
            if use_stemmer:
                word = str(porter_stemmer.stem(word))
            processed_tweet.append(word)

    return ' '.join(processed_tweet)


In [67]:
all_the_tweets = [] 
with open('output_json.json', 'w') as outfile:
        
    for tweet in tweepy.Cursor(api.search, q='#' + hashtag,rpp=100).items(maximum_number_of_tweets_to_be_extracted):
        
        if (not tweet.retweeted) :
            
            json.dump(tweet._json, outfile, indent=2) 
            outfile.write('\n')
            all_the_tweets.append(tweet)
    #with open('tweets_with_hashtag_' + hashtag + '.txt', 'a') as the_file:
        #the_file.write(str(tweet.text.encode('utf-8')) + '\n')
    #data=process_or_store(tweet._json)
    #get_tweets()
      

print ('Extracted ' + str(maximum_number_of_tweets_to_be_extracted) + ' tweets with hashtag #' + hashtag)

Extracted 10 tweets with hashtag #Datascience


In [68]:
outtweets = [[tweet.id_str, tweet.created_at,tweet.text.encode('utf-8'),tweet.user.name,tweet.user.screen_name,tweet.user.friends_count,tweet.user.followers_count,tweet.retweet_count,tweet.favorite_count] for tweet in all_the_tweets]
  
        

In [69]:
with open('_tweets.csv', 'w', encoding='utf8') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'created_at', 'text','user','screen_name','friends_count','followers_count','retweet_count','favorite_count'])
    
    writer.writerows(outtweets)

In [70]:
import numpy as np
import pandas as pd

dataset=pd.read_csv('_tweets.csv',encoding="ISO-8859-1")


In [71]:
dataset.head()

,id,created_at,text,user,screen_name,friends_count,followers_count,retweet_count,favorite_count
0,1114756842746523648,2019-04-07 05:08:37,b'Artificial intelligence in Australia needs t...,"Iain Brown, PhD",IainLJBrown,146618,148392,1,0
1,1114756799964557312,2019-04-07 05:08:26,"b""RT @chidambara09: See Someone's Facebook and...",TechnoJeder IoT,IoT_Technojeder,29,717,2,0
2,1114756760252715009,2019-04-07 05:08:17,b'Welcome to my blog!\nhttps://t.co/EUfeo0ESwR...,Makoto Kyougoku,mk76e,361,215,0,0
3,1114756587070152704,2019-04-07 05:07:36,b'32 Statistical Concepts Explained in Simple ...,Daniel Bastos ð§ð·,danielbastos,1259,1899,0,0
4,1114756500256374784,2019-04-07 05:07:15,"b""RT @chidambara09: See Someone's Facebook and...",Chris Chua,theChrisChua,222,1939,2,0


In [74]:
def extract_hash_tags(s):
    my_set=set(part[1:] for part in s.split() if part.startswith('#'))
    my_list=list(my_set)
    return(my_list)

#dataset['Hashtags']=dataset['text'].apply(lambda row: extract_hash_tags(row))

#x=dataset['text'].apply(lambda row: extract_hash_tags(row))
#x

In [75]:
for list in x:
    for x in list:
        #z=set()
        #z=z.union(i)
        print(x)
    


a


In [76]:
print(z)

set()


In [77]:
s={1,2}
set_1={3,4}

In [78]:
def equal_ignore_order(a, b):
    """ Use only when elements are neither hashable nor sortable! """
    unmatched = list(b)
    for element in a:
        try:
            unmatched.remove(element)
        except ValueError:
            return False
    return not unmatched

In [80]:
#dataset=dataset.drop(['text_1'],axis=1)

In [81]:
def remove_hashtags(x):
    if '#' in x:
        return str(x.replace('#',''))
        
dataset['TextNoHashtags']=dataset['text'].apply(remove_hashtags)

In [82]:
dataset.head()

,id,created_at,text,user,screen_name,friends_count,followers_count,retweet_count,favorite_count,TextNoHashtags
0,1114756842746523648,2019-04-07 05:08:37,b'Artificial intelligence in Australia needs t...,"Iain Brown, PhD",IainLJBrown,146618,148392,1,0,None
1,1114756799964557312,2019-04-07 05:08:26,"b""RT @chidambara09: See Someone's Facebook and...",TechnoJeder IoT,IoT_Technojeder,29,717,2,0,None
2,1114756760252715009,2019-04-07 05:08:17,b'Welcome to my blog!\nhttps://t.co/EUfeo0ESwR...,Makoto Kyougoku,mk76e,361,215,0,0,b'Welcome to my blog!\nhttps://t.co/EUfeo0ESwR...
3,1114756587070152704,2019-04-07 05:07:36,b'32 Statistical Concepts Explained in Simple ...,Daniel Bastos ð§ð·,danielbastos,1259,1899,0,0,b'32 Statistical Concepts Explained in Simple ...
4,1114756500256374784,2019-04-07 05:07:15,"b""RT @chidambara09: See Someone's Facebook and...",Chris Chua,theChrisChua,222,1939,2,0,None


In [139]:
dataset['Hashtags'].unique()

TypeError: unhashable type: 'set'

In [110]:
''''with open('output_json_filtered.json', 'w') as outfile2:
        filtered_keys = ["tweet.created_at", "tweet.id", "tweet.user.followers_count", "tweet.user.friends_count", "tweet.retweet_count", "tweet.favorite_count", "tweet.text"]
        for filtered_keys in tweet:
            json.dump(filtered_keys._json, outfile2, indent=2) 
            outfile2.write('\n')

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-110-222f4522150f>, line 5)

citation:
https://github.com/abdulfatir/twitter-sentiment-analysis/blob/master/code/preprocess.py

In [4]:
mystring="b'Artificial intelligence in Australia needs t"

In [6]:
str(mystring, 'utf-8')

TypeError: decoding str is not supported